<a href="https://colab.research.google.com/github/Egozit/ds_master_dataton/blob/main/02_team_hak_19_10_data_gathering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
import numpy as np
import requests 
import re
from geopy.geocoders import Nominatim 
import json
from lxml import etree, objectify
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
import statistics
from geopy.distance import distance

## Functions

In [2]:
def html_table_read(html_table, to_erase=None):
    """
    This funtion transform BeautifulSoup tag object with table
    into list of columns. This version removes references from
    column names only.
    TODO: 1. Add references check
    TODO: 2. Refactoring is needed
    :param bs4.tag html_table:
    :param list to_erase: [optional] list of symbols which must
                                     be removed from the results
                                     (Default = [])
    :return: list (Content of the table),
             list (Content of the column row)
    """

    if to_erase is None:
        to_erase = []
    column_names = []
    table_content = []

    html_rows = html_table.find_all('tr')
    raw_column_names = html_rows[0].find_all('th')

    for element in raw_column_names:
        if element.find('sup') is not None:
            element.sup.decompose()
        name = re.sub("<.*?>", "", str(element))
        column_names.append(name)

    for row in html_rows:
        if row.td is not None:
            current_row = []
            row_elements = row.find_all('td')
            for element in range(0, len(row_elements)):
                value = re.sub("<.*?>", "", str(row_elements[element]))
                for symbol in to_erase:
                    value = value.replace(symbol, "")
                current_row.append(value)
            table_content.append(current_row)
    return (table_content, column_names)


def get_cov_area_tag_df(html_table):
    """
    This function parses data from html file into two-dimensional list.
    """
    table = []
    tag_list = html_table.find(
        'div', style="text-align: justify;").find_all('td')

    for tag in tag_list:
        table.append(tag.span.text)

    num_el = 7
    table = [table[i:i + num_el] for i in range(0, len(table), num_el)]
    return table


def transliterate(name):
    """
    Author: LarsKort
    https://gist.github.com/ledovsky/6398962
    Date: 16/07/2011; 1:05 GMT-4;
    This function replaces Cyrillic characters with the corresponding 
    Latin characters.
    """
    # Dictionary with replacements
    replacement = {'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e',
              'ё': 'e', 'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k',
              'л': 'l', 'м': 'm', 'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r',
              'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'h', 'ц': 'c',
              'ч': 'cz', 'ш': 'sh', 'щ': 'scz', 'ъ': '', 'ы': 'y', 'ь': '',
              'э': 'e', 'ю': 'u', 'я': 'ja', 'А': 'A', 'Б': 'B', 'В': 'V',
              'Г': 'G', 'Д': 'D', 'Е': 'E', 'Ё': 'E', 'Ж': 'ZH', 'З': 'Z',
              'И': 'I', 'Й': 'I', 'К': 'K', 'Л': 'L', 'М': 'M', 'Н': 'N',
              'О': 'O', 'П': 'P', 'Р': 'R', 'С': 'S', 'Т': 'T', 'У': 'U',
              'Ф': 'F', 'Х': 'H', 'Ц': 'C', 'Ч': 'CZ', 'Ш': 'SH', 'Щ': 'SCH',
              'Ъ': '', 'Ы': 'y', 'Ь': '', 'Э': 'E', 'Ю': 'U', 'Я': 'YA',
              'ґ': '', 'ї': '', 'є': '', 'Ґ': 'g', 'Ї': 'i', 'Є': 'e', 
              '«':'"','»':'"'}

    # Replacing every character from the dictionary
    for key in replacement:
        name = name.replace(key, replacement[key])
    return name


def replace_symbol_in_columns(dataframe, lst_columns, find_sym, new_sym):
    """
    This function replaces symbol find_sym with new_sym in dataframe 
    in columns specified in lst_columns 
    """
    for col in lst_columns:
        dataframe[col] = dataframe[col].apply(
            lambda x: re.sub(find_sym, new_sym, str(x)))

def contat_unique(x, sep=', ', lowercase=False):
    if lowercase == True:
        x = x.str.lower()
    return set(sep.join(x).split(sep))

def set_to_str(x):
    return str(x).replace("'",'').strip('{}')

def series_to_list(df):
    return df.values.tolist()

def add_labels(x, st_type=False):
    if st_type == True:
        labels = temp_tpu_data.iloc[x.index]['Station_type']
        return dict(zip(x, labels))
    else:
        labels = temp_tpu_data.iloc[x.index]['global_id']
        return dict(zip(labels, x))

def find_mean_coordinate(x):
    x['center'] = np.mean(list(x.values()))
    return x

def add_new_status(x):
    if len(x)>1:
        if 'построен' in list(x.values()):
            x = 'частично построен'
        else:
            x = 'строится'
    else:
        x = list(x.values())[0]
    return x

def return_max_year(x):
    x = max(list(x.values()))
    return x

def add_global_id(df, col):
    temp_col = df[col].apply(lambda x : abs(hash(x)))
    if len(df[col]) == len(temp_col.unique()):
        return temp_col
    else:
        return add_random(col)
    
def add_random(col):
    length = len(col)
    new_col = col + np.random.randint(0, length**2, length)
    if len(col) == len(new_col.unique()):
        return new_col
    else:
        return add_random(new_col)

def boundaries(arr, val, rad):
    arr = np.array(arr)
    idx_min = np.argmin(abs(arr-(val-rad)))
    idx_max = np.argmin(abs(arr-(val+rad)))
    return idx_min, idx_max


def count_dist(x_arr, p_lat, p_lon):
    dist_l = []
    for i in x_arr:
        dist = distance([i[1], i[2]], [p_lat, p_lon]).m
        if dist == 0:
            return i[0], dist
        else:
            dist_l.append([i[0], dist])
    dist_l.sort(key=lambda x: x[1])
    return dist_l[0]

def get_linesname(html_table):
    """
    This function parses Moscow metro lines numbers
    from html file into a list.
    """
    table = []
    for tag in soup.find_all('td'):
        if tag.find('div', {'class': 'oneline'}):
            table.append([tag.div.div.get('id').replace('line', ''), 
                          tag.div.text])
 
    return table


def parse_passenger_turnover(url_part):
    """
    This function parses passenger turnover for each 
    station for every line: url_part.
    """
    main_domain_url = 'https://www.metro-msk.ru/stancii-metro/liniya-' + url_part + '/'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'}

    coverage_area_response = requests.get(main_domain_url, headers=headers).text
    soup = BeautifulSoup(coverage_area_response, 'html.parser')

    passenger_tr = []

    for tag in soup.find_all('td'):
        found = tag.find('div', id='linesname')
        if found:   
            key = found.parent.find_next('td').find('a')
            if not key:
                continue
            key = key.text
            value = found.parent.find_next('td').find('font')
            try:
                value = value.text
            except AttributeError:
                value = ''
            passenger_tr.append([key, value])

    return passenger_tr


def clean_passenger_tr_df(raw_passenger_turnover):
    '''
    This function cleans parsed passenger turnover data
    and returns a tidy dataframe 
    '''
    station_list, turnover_list, line_list = [], [], []
    for el in raw_passenger_turnover:
        for line, stations in el.items():
            for station in stations:
                station_list.append(station[0])
                turnover_list.append(station[1])
                line_list.append(line)
     
    passenger_turnover = pd.DataFrame(list(zip(line_list, station_list, 
                                               turnover_list)), 
                                               columns=['line','station', 
                                               'passengers_per_day'])

    symbol_to_replace = {
        ' чел. в сутки' : '',
        'тыс.' : '',
        ' ' : '',
        '\.' : '',
        '\t' : '',
    }

    for key, value in symbol_to_replace.items():
        replace_symbol_in_columns(passenger_turnover, ['passengers_per_day'], 
                                  key, value)

    passenger_turnover['passengers_per_day'] = pd.to_numeric(
        passenger_turnover['passengers_per_day'], errors='coerce'
        )
    passenger_turnover.drop(passenger_turnover.loc[passenger_turnover['line'] ==
                                                   'МиниметроВыставочная, ' + 
                                                   'Международная'].index, 
                                                    inplace=True)
                                                   
    return passenger_turnover         

# Data preparation


## Moscow districts data

In [3]:
# Load Moscow districts table from Wikipedia
target_url = ('https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%' +
               '81%D0%BE%D0%BA_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BE%' +
               'D0%B2_%D0%B8_%D0%BF%D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0' +
               '%BD%D0%B8%D0%B9_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B')

to_drop = [' ', '\t', '\n', '\xa0']

districts_response = requests.get(target_url).content.decode(encoding='utf-8')

soup = BeautifulSoup(districts_response)

dist_html_table = soup.find('table')

table_content, column_names = html_table_read(dist_html_table, to_drop)


raw_districts_df = pd.DataFrame(table_content, columns=column_names)
raw_districts_df.drop(columns=['№',
                               'Флаг',
                               'Герб',
                               'Название района/поселения[3][4]',
                               'Адми-нистра-тивныйокруг'],
                      inplace=True,
                      errors='ignore')
raw_districts_df.columns = ['object_district_name',
                            'object_district_square_m2',
                            'object_district_population',
                            'object_district_population_density',
                            'object_district_living_space_m2',
                            'object_district_living_space_m2_per_person']

# Cast data types 
raw_districts_df['object_district_population_density'] = raw_districts_df.\
                                        object_district_population_density.\
                                        astype('float64')

# Replace symbols
columns_list = ['object_district_square_m2',
                'object_district_living_space_m2',
                'object_district_living_space_m2_per_person']
for col in columns_list:
    raw_districts_df[col] = pd.to_numeric(
        raw_districts_df[col].apply(
            lambda x: re.sub(',', '.', str(x))))

raw_districts_df['object_district_population'] = pd.to_numeric(
    raw_districts_df['object_district_population'].apply(
        lambda x: re.sub('↗', ' ', str(x))))

# Convert kilometers to meters
columns_list = ['object_district_square_m2', 'object_district_living_space_m2']
raw_districts_df[columns_list] = raw_districts_df[columns_list]*1000


In [4]:
raw_districts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   object_district_name                        146 non-null    object 
 1   object_district_square_m2                   146 non-null    float64
 2   object_district_population                  146 non-null    int64  
 3   object_district_population_density          146 non-null    float64
 4   object_district_living_space_m2             125 non-null    float64
 5   object_district_living_space_m2_per_person  125 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 7.0+ KB


In [5]:
raw_districts_df.head()

,object_district_name,object_district_square_m2,object_district_population,object_district_population_density,object_district_living_space_m2,object_district_living_space_m2_per_person
0,Академический,5830.0,110038,18874.44,2467000.0,22.7
1,Алексеевский,5290.0,80634,15242.72,1607900.0,20.5
2,Алтуфьевский,3250.0,57697,17752.92,839300.0,15.5
3,Арбат,2110.0,36308,17207.58,731000.0,26.0
4,Аэропорт,4580.0,79541,17367.03,1939700.0,25.9


## Store coverage areas data

In [6]:
# Load store coverage areas table
target_url = 'http://www.arhitrade.com/education.php?Id=43'
 
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0)' +
                         'Gecko/20100101 Firefox/81.0'}
 
coverage_area_response = requests.get(target_url, headers=headers).text
soup = BeautifulSoup(coverage_area_response, 'html.parser')

raw_cov_area_df = get_cov_area_tag_df(soup)
cov_area_df = pd.DataFrame(raw_cov_area_df[1:], columns=raw_cov_area_df[0])
 
# Rename columns
cov_area_df.columns = ['shop_type', 'shop_square', 'access_time_on_foot',
                       'access_time_transport', 'max_reach_distance',
                       'real_reach_distance', 'coverage_radius']
 
# Replace symbols
symbol_to_replace = {
    '[~км]': ['max_reach_distance', 'real_reach_distance',
              'coverage_radius'],
    '[докв.минутблее]': ['shop_square', 'access_time_on_foot',
                         'access_time_transport'],
    '[ \xa0]': ['shop_square', 'access_time_on_foot',
                'access_time_transport', 'max_reach_distance',
                'real_reach_distance', 'coverage_radius']}
 
for key, value in symbol_to_replace.items():
    replace_symbol_in_columns(cov_area_df, value, key, '')
 
replace_symbol_in_columns(
    cov_area_df, ['max_reach_distance', 'coverage_radius'], ',', '.')
 
# Averaging of intervals  
for idx, el in enumerate(cov_area_df.shop_square):
    cov_area_df.shop_square[idx] = statistics.mean([*map(int, el.split('–'))])
 
# Cast data types 
columns_list = ['access_time_on_foot', 'access_time_transport', 'shop_square',
                'max_reach_distance', 'real_reach_distance', 'coverage_radius']
 
for column in columns_list:
    cov_area_df[column] = pd.to_numeric(cov_area_df[column], errors='coerce',
                                        downcast='float')
 
# Convert kilometers to meters
columns_list = ['max_reach_distance', 'real_reach_distance', 'coverage_radius']
cov_area_df[columns_list] = cov_area_df[columns_list] * 1000
 
# Add store class information
cov_area_df['shop_class'] = [0, 1, 2, 3]


In [7]:
cov_area_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   shop_type              4 non-null      object 
 1   shop_square            4 non-null      float32
 2   access_time_on_foot    4 non-null      float32
 3   access_time_transport  3 non-null      float32
 4   max_reach_distance     4 non-null      float32
 5   real_reach_distance    4 non-null      float32
 6   coverage_radius        4 non-null      float32
 7   shop_class             4 non-null      int64  
dtypes: float32(6), int64(1), object(1)
memory usage: 288.0+ bytes


In [8]:
cov_area_df.head()

,shop_type,shop_square,access_time_on_foot,access_time_transport,max_reach_distance,real_reach_distance,coverage_radius,shop_class
0,Магазин шаговой доступности (мини-маркет),300.0,15.0,NaN,1500.0,1000.0,1000.0,0
1,"Универсальный магазин (миди-маркет, универма...",450.0,30.0,7.0,3700.0,2000.0,2500.0,1
2,Средние специализированные и универсальные м...,1300.0,60.0,15.0,7700.0,4000.0,5000.0,2
3,Крупные специализированные и универсальные м...,2000.0,120.0,30.0,15000.0,10000.0,12000.0,3


## Transport hubs and stores data

In [10]:
# Load transports hubs table and stores tabels 
target_url = 'https://op.mos.ru/EHDWSREST/catalog/export/get?id=872986'
trade_place_data = pd.read_json(target_url, compression='zip',
                                encoding='windows-1251')

target_url = 'https://op.mos.ru/EHDWSREST/catalog/export/get?id=955944'
tpu_data = pd.read_csv(target_url, compression='zip',
                       encoding='windows-1251', sep=';').drop(
                           ['Unnamed: 11'], axis=1)
target_url = 'https://op.mos.ru/EHDWSREST/catalog/export/get?id=976939'
service_place_data = pd.read_json(target_url, compression='zip',
                                encoding='windows-1251').drop(
                                    columns=['Latitude_WGS84', 
                                             'Longitude_WGS84'])

# Merge stores tables
trade_place_data = trade_place_data.append(service_place_data,
                                           ignore_index=True)

# Extract phone number
trade_place_data['PublicPhone'] = trade_place_data['PublicPhone'].apply(
    lambda x: x[0]['PublicPhone'])

# Split geoData column into two: longitude and latitude
trade_place_data['geoData'] = trade_place_data['geoData'].astype(str)
trade_place_data['longitude'], trade_place_data['latitude'] = zip(
    *trade_place_data.geoData.str.strip("{'type': 'Point', 'coordinates': []}")
    .str.split(', '))

columns_list = ['latitude', 'longitude']
trade_place_data[columns_list] = trade_place_data[columns_list].astype('float')

tpu_data['tpu_longitude'], tpu_data['tpu_latitude'] = zip(
    *tpu_data.geoData.str.strip('{type=Point, coordinates=[]}')
    .str.split(', '))

columns_list = ['tpu_latitude', 'tpu_longitude']
tpu_data[columns_list] = tpu_data[columns_list].astype('float')

# Fixing errors with mixed latitude and longtitude data in TPU dataframe
geolocator = Nominatim(user_agent="demo")
address = 'Moscow, Russia'
location = geolocator.geocode(address)
mos_lat = location.latitude
mos_lng = location.longitude
for name, lat, lng in zip(tpu_data.TPUName, 
                          tpu_data.tpu_latitude, 
                          tpu_data.tpu_longitude):
    if distance([lat,lng],[mos_lat, mos_lng]).m > 200000:
        real_lat, real_lng = tpu_data[tpu_data.TPUName==name]\
                             [['tpu_longitude',
                               'tpu_latitude']].iloc[0][0],\
                             tpu_data[tpu_data.TPUName==name]\
                             [['tpu_longitude',
                               'tpu_latitude']].iloc[0][1]
        tpu_data.loc[tpu_data.TPUName==name,['tpu_longitude']] = real_lng
        tpu_data.loc[tpu_data.TPUName==name,['tpu_latitude']] = real_lat

# IsNetObject column label encoding: 0/1 - no/yes
trade_place_data.loc[trade_place_data.IsNetObject == 'нет', ['IsNetObject']] = 0
trade_place_data.loc[trade_place_data.IsNetObject == 'да', ['IsNetObject']] = 1


# Add store size information (1 - small, 2 - medium, 3 - large)
trade_place_data['PlaceSize'] = 1

trade_place_data.loc[trade_place_data.TypeObject.str.contains(
    'Супер',  case=False), 'PlaceSize'] = 2
trade_place_data.loc[trade_place_data.Name.str.contains(
    'Супермаркет', case=False), 'PlaceSize'] = 2

large_objects = ['Гипер', 
                 'Торговые центры', 
                 'Торговый Дом', 
                 'ТД ']
for obj_name in large_objects:
    trade_place_data.loc[trade_place_data.TypeObject.str.contains(
        obj_name, case=False), 'PlaceSize'] = 3
    trade_place_data.loc[trade_place_data.Name.str.contains(
        obj_name, case=False), 'PlaceSize'] = 3

# Convert District column to lowercase
tpu_data['District'] = tpu_data.District.str.replace(
    'район', '').str.strip(' ')
trade_place_data['District'] = trade_place_data.District.str.replace(
    'район', '').str.strip(' ')

# Drop excessive columns
trade_place_data.drop(columns=['ID', 'geoData'], inplace=True)
tpu_data.drop(columns=['geoData'], inplace=True)

# Delete outliners
name_list = ['Транспортно-пересадочный узел «Волоколамская» (1)',
             'Транспортно-пересадочный узел «Саларьево» (к)']
idx = np.where(tpu_data.TPUName.isin(name_list))[0]
tpu_data.drop(idx, inplace=True)

tpu_data.loc[tpu_data.TPUName =='Транспортно-пересадочный узел «Планерная» (2)',
            'TPUName'] = 'Транспортно-пересадочный узел «Планерная Химки» (2)'
tpu_data.reset_index(drop=True, inplace=True)

In [11]:
trade_place_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78086 entries, 0 to 78085
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   global_id                    78086 non-null  int64  
 1   WorkingHours                 78086 non-null  object 
 2   Name                         78086 non-null  object 
 3   IsNetObject                  78086 non-null  object 
 4   TypeService                  78086 non-null  object 
 5   TypeObject                   78086 non-null  object 
 6   AdmArea                      78086 non-null  object 
 7   District                     78086 non-null  object 
 8   Address                      78086 non-null  object 
 9   PublicPhone                  78086 non-null  object 
 10  OperatingCompany             19735 non-null  object 
 11  ClarificationOfWorkingHours  90 non-null     object 
 12  longitude                    78086 non-null  float64
 13  latitude        

In [12]:
trade_place_data.head()

,global_id,WorkingHours,Name,IsNetObject,TypeService,TypeObject,AdmArea,District,Address,PublicPhone,OperatingCompany,ClarificationOfWorkingHours,longitude,latitude,PlaceSize
0,637390942,"[{'Hours': '09:30-21:30', 'DayOfWeek': 'понеде...",Одежда. Обувь. Промтовары,0,реализация непродовольственных товаров,Магазин «Промтовары»,Северный административный округ,Беговой,"город Москва, Беговая аллея, дом 3А",(977) 757-40-37,NaN,NaN,37.564782,55.783361,1
1,637391109,"[{'Hours': '10:00-21:00', 'DayOfWeek': 'понеде...",АЙКОВЕР,0,реализация непродовольственных товаров,Магазин-салон,Центральный административный округ,Тверской,"город Москва, Новослободская улица, дом 50/1, ...",(499) 250-12-40,NaN,NaN,37.594849,55.786559,1
2,637391684,"[{'Hours': '10:00-22:00', 'DayOfWeek': 'понеде...",Магазин «Радуга»,0,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,Марьино,"город Москва, Марьинский бульвар, дом 4",(909) 977-22-02,NaN,NaN,37.749819,55.647955,1
3,637391714,"[{'Hours': '10:00-22:00', 'DayOfWeek': 'понеде...",Точка любви,1,реализация непродовольственных товаров,Прочие специализированные непродовольственные ...,Северный административный округ,Беговой,"город Москва, Ленинградский проспект, дом 12",(985) 780-51-78,Точка любви,NaN,37.578783,55.780188,1
4,637391844,"[{'Hours': 'круглосуточно', 'DayOfWeek': 'поне...",ПРОДУКТЫ,0,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,Лефортово,"город Москва, Волочаевская улица, дом 40Б",(495) 361-17-35,NaN,NaN,37.678140,55.752482,1


In [13]:
tpu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  276 non-null    int64  
 1   TPUName             276 non-null    object 
 2   global_id           276 non-null    int64  
 3   AdmArea             276 non-null    object 
 4   District            276 non-null    object 
 5   NearStation         276 non-null    object 
 6   YearOfComissioning  276 non-null    int64  
 7   Status              276 non-null    object 
 8   AvailableTransfer   276 non-null    object 
 9   CarCapacity         107 non-null    float64
 10  tpu_longitude       276 non-null    float64
 11  tpu_latitude        276 non-null    float64
dtypes: float64(3), int64(3), object(6)
memory usage: 26.0+ KB


In [14]:
tpu_data.head()

,ID,TPUName,global_id,AdmArea,District,NearStation,YearOfComissioning,Status,AvailableTransfer,CarCapacity,tpu_longitude,tpu_latitude
0,96,Транспортно-пересадочный узел «Сортировочная» (п),1769845,Юго-Восточный административный округ,Лефортово,платформа «Сортировочная»,2014,проект,пригородная железная дорога,NaN,37.720353,55.764080
1,97,Транспортно-пересадочный узел «Фрезер» (п),1769846,Юго-Восточный административный округ,Нижегородский,платформа «Фрезер»,2014,построен,пригородная железная дорога,45.0,37.738276,55.745111
2,98,Транспортно-пересадочный узел «Чухлинка» (п),1769847,Юго-Восточный административный округ,Рязанский,платформа «Чухлинка»,2014,построен,пригородная железная дорога,NaN,37.764119,55.733620
3,99,Транспортно-пересадочный узел «Братиславская» (п),1769848,Юго-Восточный административный округ,Марьино,станция метро «Братиславская»,2015,построен,метро; перехватывающая парковка,320.0,37.750696,55.659641
4,100,Транспортно-пересадочный узел «Волжская» (п),1769849,Юго-Восточный административный округ,Люблино,станция метро «Волжская»,2014,построен,метро; перехватывающая парковка,240.0,37.755894,55.690307


## Passenger turnover data

In [15]:
linename_url = 'https://www.metro-msk.ru/stat/2019/'
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0)' + 
           'Gecko/20100101 Firefox/81.0'}

coverage_area_response = requests.get(linename_url, headers=headers).text
soup = BeautifulSoup(coverage_area_response, 'html.parser')

raw_linesname = get_linesname(soup)

raw_passenger_turnover = []
for url_part, line in raw_linesname:
    raw_passenger_turnover.append({line : parse_passenger_turnover(url_part)})

passenger_turnover = clean_passenger_tr_df(raw_passenger_turnover)

In [16]:
passenger_turnover.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 0 to 228
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   line                229 non-null    object 
 1   station             229 non-null    object 
 2   passengers_per_day  215 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.2+ KB


In [17]:
passenger_turnover.head()

,line,station,passengers_per_day
0,Таганско-Краснопресненская,Планерная,56200.0
1,Таганско-Краснопресненская,Сходненская,83200.0
2,Таганско-Краснопресненская,Тушинская,42400.0
3,Таганско-Краснопресненская,Спартак,21000.0
4,Таганско-Краснопресненская,Щукинская,90700.0


## Transport hub & Passenger turnover data merging

In [18]:
temp_df = tpu_data.TPUName.str.extract(r'(.*\«.*?») \((.*?)\)')
temp_df = temp_df.rename(columns = 
                         {0:'Name',
                          1:'Station_type'}
                        )
tpu_data_copy = tpu_data.copy()
temp_tpu_data = tpu_data.join(temp_df)


tpu_data = temp_tpu_data.groupby('Name', as_index=False).agg({
                              'global_id': lambda x: add_labels(x,
                                                                st_type=True),
                              'AdmArea': lambda x: add_labels(x),
                              'District': lambda x: add_labels(x),
                              'NearStation': lambda x: set_to_str(
                                  contat_unique(x)),
                              'YearOfComissioning':lambda x: add_labels(x),
                              'Status': lambda x: add_labels(x),
                              'AvailableTransfer': lambda x: set_to_str(
                                  contat_unique(x, sep='; ', lowercase=True )),
                              'CarCapacity': sum,
                              'tpu_longitude': lambda x: add_labels(x),
                              'tpu_latitude': lambda x: add_labels(x)})
tpu_data.CarCapacity.replace(0.0, np.nan, inplace=True)

tpu_data.tpu_longitude = tpu_data.tpu_longitude.apply(
    lambda x: find_mean_coordinate(x))
tpu_data.tpu_latitude = tpu_data.tpu_latitude.apply(
    lambda x: find_mean_coordinate(x))

tpu_data.YearOfComissioning = tpu_data.YearOfComissioning.apply(
    lambda x: return_max_year(x))
tpu_data.Status = tpu_data.Status.apply(lambda x: add_new_status(x))

substr = r'станция метро \«(.*?)\»'
tpu_data['station'] = tpu_data['NearStation'].str.extract(substr)
new_passenger_turnover  = passenger_turnover.groupby(
    'station', as_index=False).agg({
        'line' : lambda x: set_to_str(contat_unique(x)),
        'passengers_per_day' : sum })
tpu_data = pd.merge(tpu_data, new_passenger_turnover,  
                   on='station', 
                   how='left')
tpu_data.rename(columns = {'global_id' : 'tpu_content',
                          'station' : 'subway_station',
                          'Name' : 'TPUName'}, inplace=True)

tpu_data.insert(2, 'tpu_content_count', tpu_data.tpu_content.apply(len))
tpu_data.insert(1, 'global_id', add_global_id(tpu_data, 'TPUName'))

In [19]:
tpu_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 206
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TPUName             207 non-null    object 
 1   global_id           207 non-null    int64  
 2   tpu_content         207 non-null    object 
 3   tpu_content_count   207 non-null    int64  
 4   AdmArea             207 non-null    object 
 5   District            207 non-null    object 
 6   NearStation         207 non-null    object 
 7   YearOfComissioning  207 non-null    int64  
 8   Status              207 non-null    object 
 9   AvailableTransfer   207 non-null    object 
 10  CarCapacity         97 non-null     float64
 11  tpu_longitude       207 non-null    object 
 12  tpu_latitude        207 non-null    object 
 13  subway_station      159 non-null    object 
 14  line                131 non-null    object 
 15  passengers_per_day  131 non-null    float64
dtypes: float

In [20]:
tpu_data.head()

,TPUName,global_id,tpu_content,tpu_content_count,AdmArea,District,NearStation,YearOfComissioning,Status,AvailableTransfer,CarCapacity,tpu_longitude,tpu_latitude,subway_station,line,passengers_per_day
0,Транспортно-пересадочный узел «Авиамоторная»,7423989657863085847,{1770036: 'к'},1,{1770036: 'Юго-Восточный административный округ'},{1770036: 'Лефортово'},станция метро «Авиамоторная»,2019,проект,"перехватывающая парковка, станция железной дор...",290.0,"{1770036: 37.717212999987, 'center': 37.717212...","{1770036: 55.751630999933, 'center': 55.751630...",Авиамоторная,Калининская,399000.0
1,Транспортно-пересадочный узел «Автозаводская»,3851741139439695697,{1769892: 'п'},1,{1769892: 'Южный административный округ'},{1769892: 'Даниловский'},станция метро «Автозаводская»,2014,построен,"московская кольцевая железная дорога, метро",NaN,"{1769892: 37.657222151552, 'center': 37.657222...","{1769892: 55.706949489906, 'center': 55.706949...",Автозаводская,Замоскворецкая,86600.0
2,Транспортно-пересадочный узел «Академическая»,3704362173907811625,"{1769865: 'п', 170429968: 'к'}",2,{1769865: 'Юго-Западный административный округ...,"{1769865: 'Академический', 170429968: 'Академи...",станция метро «Академическая»,2018,частично построен,"перехватывающая парковка, метро",260.0,"{1769865: 37.573083937002, 170429968: 37.57341...","{1769865: 55.687492147998, 170429968: 55.68773...",Академическая,Калужско-Рижская,47700.0
3,Транспортно-пересадочный узел «Алабушево»,4475050148001869354,{1769942: 'п'},1,{1769942: 'Зеленоградский административный окр...,{1769942: 'Силино'},платформа «Алабушево»,2014,проект,пригородная железная дорога,NaN,"{1769942: 37.1399527786, 'center': 37.1399527786}","{1769942: 56.008723123693, 'center': 56.008723...",NaN,NaN,NaN
4,Транспортно-пересадочный узел «Алексеевская»,5531869237941398190,{1770013: 'п'},1,{1770013: 'Северо-Восточный административный о...,{1770013: 'Останкинский'},станция метро «Алексеевская»,2014,построен,"перехватывающая парковка, метро",390.0,"{1770013: 37.636681795289, 'center': 37.636681...","{1770013: 55.80891609238, 'center': 55.8089160...",Алексеевская,Калужско-Рижская,29200.0


## Table: Distance between store and transport hub

In [21]:
# Calculate distance from store to the nearest transport hub
tpu_tradeplace_dist_list = []
rad = 0.05

lat_col = tpu_data['tpu_latitude'].apply(lambda x: x['center'])
lon_col = tpu_data['tpu_longitude'].apply(lambda x: x['center'])

temp_tpu_lat = list(zip(tpu_data['global_id'],
                        lat_col,
                        lon_col))
temp_tpu_lat.sort(key=lambda x: x[1])

for row in range(len(trade_place_data)):
    place_id = trade_place_data.iloc[row]['global_id']
    p_lat, p_lon = trade_place_data.iloc[row][['latitude', 'longitude']]
    min_l, max_l = boundaries([x[1] for x in temp_tpu_lat], p_lat, rad)
    tp_id = trade_place_data.iloc[row]['global_id']
    if min_l != max_l:
        temp_tpu_lon = temp_tpu_lat[min_l:max_l]
        temp_tpu_lon.sort(key=lambda x: x[2])
        min_l, max_l = boundaries([x[2] for x in temp_tpu_lon], p_lon, rad)
        if min_l != max_l:
            temp_tpu_lon = temp_tpu_lon[min_l:max_l]
            tpu_tradeplace_dist_list.append([place_id] +
                                            count_dist(
                                                 temp_tpu_lon, p_lat, p_lon))
        else:
            tpu_tradeplace_dist_list.append([place_id, temp_tpu_lon[min_l][0],
                                             distance([temp_tpu_lon[min_l][1],
                                                       temp_tpu_lon[min_l][2]],
                                                      [p_lat, p_lon]).m])
    else:
        tpu_tradeplace_dist_list.append([place_id, temp_tpu_lat[min_l][0],
                                        distance([temp_tpu_lat[min_l][1],
                                                  temp_tpu_lat[min_l][2]],
                                                 [p_lat, p_lon]).m])
columns_list = ['trade_place_global_id', 'tpu_global_id', 'distance']
tpu_tradeplace_dist = pd.DataFrame(tpu_tradeplace_dist_list,
                                   columns=columns_list)

In [22]:
tpu_tradeplace_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78086 entries, 0 to 78085
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trade_place_global_id  78086 non-null  int64  
 1   tpu_global_id          78086 non-null  int64  
 2   distance               78086 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.8 MB


In [23]:
tpu_tradeplace_dist.head()

,trade_place_global_id,tpu_global_id,distance
0,637390942,5970351464247905946,947.329127
1,637391109,1297575275508927000,794.746640
2,637391684,8720156415891978423,447.344507
3,637391714,6044955882543811283,418.756267
4,637391844,9061517819343448243,676.612092


# Final dataset merging

In [54]:
rent_cost_df = pd.read_excel('https://github.com/Egozit/ds_master_dataton' +
                             '/raw/main/data/realty_cost.xlsx', header=1)

main_df = pd.merge(trade_place_data, tpu_tradeplace_dist, 
                   left_on = 'global_id', right_on = 'trade_place_global_id')

copy_tpu_data = tpu_data.copy()        
copy_tpu_data.columns = ['tpu_' + str(col) if col!='TPUName' else col for col \
                         in copy_tpu_data.columns]

# Data merging
main_df = pd.merge(main_df, copy_tpu_data,  
                   on='tpu_global_id', 
                   how='left', suffixes=['', '_y'])
main_df = pd.merge(main_df, cov_area_df[['shop_class','real_reach_distance']],  
                   left_on='PlaceSize', right_on='shop_class', 
                   how='left', suffixes=['', '_y'])
main_df = pd.merge(main_df, raw_districts_df,  
                   left_on='District', right_on='object_district_name', 
                   how='left', suffixes=['', '_y'])
main_df = pd.merge(main_df, rent_cost_df,  
                   left_on='District', right_on='Район', 
                   how='left', suffixes=['', '_y'])

# Dropping technical columns after merging
main_df = main_df.drop(columns=['trade_place_global_id', 'tpu_AdmArea',
                                'shop_class', 'object_district_name', 'Район'])

# Addind indicator to show if shop is in reach distance of transport hub 
main_df.insert(loc=3, column='IsInRange', 
               value=[0 for x in range(main_df.shape[0])])
main_df.loc[main_df.distance <= main_df.real_reach_distance, \
            ['IsInRange']] = 1

# Adjusting column names
main_df.rename(columns={
      'global_id': 'object_global_id',
      'Name':'object_name',
      'IsNetObject':'is_network_object',
      'IsInRange':'is_tpu_in_coverage',
      'OperatingCompany':'object_operating_company',
      'TypeService':'object_service_type',
      'TypeObject':'object_type',
      'AdmArea':'object_area',
      'District':'object_district',
      'Address':'object_address',
      'PublicPhone':'object_phone',
      'WorkingHours':'object_working_hours',
      'ClarificationOfWorkingHours':'object_working_hours_clarification',
      'PlaceSize':'object_size',
      'longitude':'object_longitude',
      'latitude':'object_latitude',
      'TPUName':'tpu_name',
      'tpu_District':'tpu_district',
      'tpu_NearStation':'tpu_near_station',
      'tpu_YearOfComissioning':'tpu_comissioning_year',
      'tpu_Status':'tpu_status',
      'tpu_AvailableTransfer':'tpu_available_transfer',
      'tpu_CarCapacity':'tpu_car_capacity',
      'real_reach_distance':'object_real_reach_distance_t',
      'distance': 'distance_to_tpu',
      'tpu_tpu_latitude': 'tpu_latitude', 
      'tpu_line' : 'subway_line',
      'tpu_subway_station' : 'subway_station',
      'tpu_passengers_per_day' : 'subway_passengers_per_day',
      'tpu_tpu_longitude': 'tpu_longitude',
      'tpu_tpu_content' : 'tpu_content',
      'tpu_tpu_content_count' : 'tpu_content_count',
      'Отдельно стоящие здания (м2)': 'object_district_building_property_' +
                                      'price_per_m2',
      'Торговые помещения (м2)': 'object_district_tradeplace_property_' + 
                                'price_per_m2',
      'Помещения свободного назначения (м2)': 'object_district_generalplace_' +
                                              'property_price_per_m2',
      'Отдельно стоящие здания (м2).1': 'object_district_building_rent_' + 
                                        'price_per_m2',
      'Торговые помещения (м2).1': 'object_district_tradeplace_rent_' + 
                                   'price_per_m2',
      'Помещения свободного назначения (м2).1': 'object_district_' + 
                                                'generalplace_' +
                                                'rent_price_per_m2'}, 
      inplace=True)

coordinates = ['object_latitude',
               'object_longitude']
for coordinate in coordinates:
    main_df[coordinates] = main_df[coordinates].astype('float64')

main_df.insert(loc=16, column='object_real_reach_distance', 
               value=main_df.object_real_reach_distance_t)
main_df = main_df.drop(columns=['object_real_reach_distance_t'])  

# Casting tpu_available_transfer column to lower case
main_df['tpu_available_transfer'] = main_df.tpu_available_transfer.str.lower()

# Fixing some types to int32
main_df.is_network_object = main_df.is_network_object.astype('int32')
main_df.distance_to_tpu = main_df.distance_to_tpu.astype('int32')

mask = main_df['subway_passengers_per_day']==0.0, ['subway_passengers_per_day']
main_df.loc[mask] = np.nan

# Changing columns position
col_names = ['object_working_hours', 'is_tpu_in_coverage',
             'object_operating_company', 'object_size',
             'distance_to_tpu', 'tpu_name']
new_position = [11, 3, 4, 13, 17, 18]

for col_name, pos in zip(col_names, new_position):
    col = main_df.pop(col_name)
    main_df.insert(pos, col.name, col)

In [56]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78086 entries, 0 to 78085
Data columns (total 44 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   object_global_id                                    78086 non-null  int64  
 1   object_name                                         78086 non-null  object 
 2   is_network_object                                   78086 non-null  int32  
 3   is_tpu_in_coverage                                  78086 non-null  int64  
 4   object_operating_company                            19735 non-null  object 
 5   object_service_type                                 78086 non-null  object 
 6   object_type                                         78086 non-null  object 
 7   object_area                                         78086 non-null  object 
 8   object_district                                     78086 non-null  object 


In [26]:
main_df.head()

,object_global_id,object_working_hours,object_name,is_tpu_in_coverage,is_network_object,object_service_type,object_type,object_area,object_district,object_address,object_phone,object_operating_company,object_working_hours_clarification,object_longitude,object_latitude,object_size,object_real_reach_distance,tpu_global_id,distance_to_tpu,tpu_name,tpu_content,tpu_content_count,tpu_district,tpu_near_station,tpu_comissioning_year,tpu_status,tpu_available_transfer,tpu_car_capacity,tpu_longitude,tpu_latitude,subway_station,subway_line,subway_passengers_per_day,object_district_square_m2,object_district_population,object_district_population_density,object_district_living_space_m2,object_district_living_space_m2_per_person,object_district_building_property_price_per_m2,object_district_tradeplace_property_price_per_m2,object_district_generalplace_property_price_per_m2,object_district_building_rent_price_per_m2,object_district_tradeplace_rent_price_per_m2,object_district_generalplace_rent_price_per_m2
0,637390942,"[{'Hours': '09:30-21:30', 'DayOfWeek': 'понеде...",Одежда. Обувь. Промтовары,1,0,реализация непродовольственных товаров,Магазин «Промтовары»,Северный административный округ,Беговой,"город Москва, Беговая аллея, дом 3А",(977) 757-40-37,NaN,NaN,37.564782,55.783361,1,2000.0,5970351464247905946,947,Транспортно-пересадочный узел «Петровский парк»,{1770010: 'п'},1,{1770010: 'Аэропорт'},станция метро «Петровский парк»,2015,проект,"перехватывающая парковка, метро",300.0,"{1770010: 37.556888576359, 'center': 37.556888...","{1770010: 55.790614556397, 'center': 55.790614...",Петровский парк,"Солнцевская, Каховская",NaN,5560.0,42908.0,7717.27,791100.0,18.8,NaN,426341.0,378271.0,NaN,3031.0,2500.0
1,637391109,"[{'Hours': '10:00-21:00', 'DayOfWeek': 'понеде...",АЙКОВЕР,1,0,реализация непродовольственных товаров,Магазин-салон,Центральный административный округ,Тверской,"город Москва, Новослободская улица, дом 50/1, ...",(499) 250-12-40,NaN,NaN,37.594849,55.786559,1,2000.0,1297575275508927000,794,Транспортно-пересадочный узел «Новослободская»,{1769880: 'п'},1,{1769880: 'Тверской'},станция метро «Новослободская»,2014,построен,метро,39.0,"{1769880: 37.600518160589, 'center': 37.600518...","{1769880: 55.780175380754, 'center': 55.780175...",Новослободская,Кольцевая,49000.0,7270.0,77947.0,10721.73,1253300.0,16.4,285469.0,987610.0,664251.0,2019.0,4923.0,4050.0
2,637391684,"[{'Hours': '10:00-22:00', 'DayOfWeek': 'понеде...",Магазин «Радуга»,1,0,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,Марьино,"город Москва, Марьинский бульвар, дом 4",(909) 977-22-02,NaN,NaN,37.749819,55.647955,1,2000.0,8720156415891978423,447,Транспортно-пересадочный узел «Марьино»,{1769856: 'к'},1,{1769856: 'Марьино'},станция метро «Марьино»,2018,проект,"перехватывающая парковка, метро",230.0,"{1769856: 37.744333999899, 'center': 37.744333...","{1769856: 55.650510000027, 'center': 55.650510...",Марьино,Люблинская,181900.0,11910.0,254142.0,21338.54,4523000.0,18.6,NaN,416755.0,110717.0,NaN,3493.0,1580.0
3,637391714,"[{'Hours': '10:00-22:00', 'DayOfWeek': 'понеде...",Точка любви,1,1,реализация непродовольственных товаров,Прочие специализированные непродовольственные ...,Северный административный округ,Беговой,"город Москва, Ленинградский проспект, дом 12",(985) 780-51-78,Точка любви,NaN,37.578783,55.780188,1,2000.0,6044955882543811283,418,Транспортно-пересадочный узел «Белорусский»,{170430036: 'к'},1,{170430036: 'Тверской'},Белорусский вокзал,2020,проект,"перехватывающая парковка, станция железной дороги",NaN,"{170430036: 37.58183100001, 'center': 37.58183...","{170430036: 55.776841999905, 'center': 55.7768...",NaN,NaN,NaN,5560.0,42908.0,7717.27,791100.0,18.8,NaN,426341.0,378271.0,NaN,3031.0,2500.0
4,637391844,"[{'Hours': 'круглосуточно', 'DayOfWeek': 'поне...",ПРОДУКТЫ,1,0,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,Лефортово,"город Москва, Волочаевская улица, дом 40Б",(495) 361-17-35,N

# Saving dataset example in JSON format

In [27]:
# Top 5 rows to show dataset structure
result = main_df.head(5).to_json(orient='index', 
                                 force_ascii=False).encode('utf8')
parsed = json.loads(result)
with open('data_example.json', 'w', encoding='utf8') as json_file:
    json.dump(parsed, json_file, ensure_ascii=False, indent=4)

# Saving the final dataset

In [57]:
#compression_opts = dict(method='zip', archive_name='final_dataset.csv') 
#main_df.to_csv('final_dataset.zip', compression=compression_opts)
main_df.to_csv('final_dataset.csv', sep=';', encoding='cp1251')
tpu_data.to_csv('transport_hub.csv', sep=';', encoding='cp1251')